<a href="https://colab.research.google.com/github/Salma-Kassem/DeepLearning/blob/main/_Yolo3k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

yolo3

In [ ]:
!pip install tensorflow

In [ ]:
!git clone https://github.com/xiaochus/YOLOv3.git

##Make training data directories
!mkdir yolo3_tiny

%cd yolo3_tiny

# Make image input and output directories
!mkdir images
!mkdir out

!cp ../YOLOv3/images/test/*.jpg images

!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

# copy classes file to yolo3_tiny directory
!cp ../YOLOv3/data/coco_classes.txt coco_classes.txt

#Get custom fonts for image annotations
!wget -O font.zip https://fonts.google.com/download?family=Ubuntu
!unzip font.zip -d fonts

Cloning into 'YOLOv3'...
remote: Enumerating objects: 95, done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 95 (from 1)
Receiving objects: 100% (95/95), 13.89 MiB | 17.62 MiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/yolo3_tiny/yolo3_tiny
--2025-12-10 15:10:53--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1915 (1.9K) [text/plain]
Saving to: ‘yolov3-tiny.cfg’

yolov3-tiny.cfg     100%[===================>]   1.87K  --.-KB/s    in 0s      

2025-12-10 15:10:53 (25.5 MB/s) - ‘yolov3-tiny.cfg’ saved [1915/1915]

--2025-12-10 15:10:53--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 172.67.185.199, 104.21

# Initialization

In [ ]:
import numpy as np
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, Reshape, LeakyReLU, MaxPooling2D, UpSampling2D, Lambda, concatenate
from keras.models import Model
from keras.regularizers import l2
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageDraw, ImageFont

# ML Cloud serving imports
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl, utils
from functools import partial


In [ ]:
# print("Keras version " + keras.__version__)
print("Tensorflow version" + tf.__version__) #should be 1.15.3
!python --version #3.6.9 works

Tensorflow version2.19.0
Python 3.12.12


In [ ]:
LABELS = ['raccoon', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# LABELS = ['person']

IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.3#0.5
NMS_THRESHOLD    = 0.3#0.45
ANCHORS          = [10,14, 23,27, 37,58, 81,82, 135,169, 344,319]

NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [ ]:
# wt_path = 'yolov2.weights'
# wt_path = 'yolov3.weights'
wt_path = 'yolov3-tiny.weights'
train_image_folder = './data/coco/train/'
train_annot_folder = './data/coco/trainann/'
valid_image_folder = './data/coco/val/'
valid_annot_folder = './data/coco/valann/'

# Construct the network - Model

In [ ]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [ ]:
def setWeights(filters, size, count, input_tensor, weights_file, doBatch):
    """
    Load weights from Darknet .weights file and prepare for Keras Conv2D and BatchNormalization layers.

    Args:
        filters: number of output filters for Conv2D
        size: kernel size
        count: current read position in the weights file
        input_tensor: input Keras tensor (used to get input channels)
        weights_file: open file object for darknet weights
        doBatch: whether to apply batch normalization

    Returns:
        conv_weights: list of Conv2D weights (and bias if no BN)
        bn_weight_list: list of BN weights [gamma, beta, mean, var] if BN, else empty
        count: updated count
    """
    import numpy as np
    from tensorflow.keras import backend as K

    # Get previous layer channels
    prev_layer_shape = K.int_shape(input_tensor)
    in_channels = prev_layer_shape[-1]

    # Shapes
    weights_shape = (size, size, in_channels, filters)
    darknet_w_shape = (filters, in_channels, size, size)
    weights_size = np.product(weights_shape)

    # Read convolution bias
    conv_bias = np.frombuffer(weights_file.read(filters * 4), dtype='float32')
    count += filters

    conv_weights = []
    bn_weight_list = []

    # Read batch norm weights if applicable
    if doBatch:
        bn_weights = np.frombuffer(weights_file.read(3 * filters * 4), dtype='float32')
        bn_weights = bn_weights.reshape((3, filters))
        count += 3 * filters

        # Keras BN expects [gamma, beta, running_mean, running_var]
        bn_weight_list = [
            bn_weights[0],   # gamma (scale)
            conv_bias,       # beta (shift)
            bn_weights[1],   # running mean
            bn_weights[2]    # running var
        ]

    # Read convolutional weights
    conv_weights = np.frombuffer(weights_file.read(weights_size * 4), dtype='float32')
    count += weights_size
    conv_weights = conv_weights.reshape(darknet_w_shape)
    conv_weights = np.transpose(conv_weights, (2, 3, 1, 0))  # darknet -> keras format

    # Pack weights for Keras layer
    if doBatch:
        conv_weights = [conv_weights]  # BN handles bias
    else:
        conv_weights = [conv_weights, conv_bias]  # No BN: include bias

    return conv_weights, bn_weight_list, count


In [ ]:

weights_file = open(wt_path, 'rb')
weights_header = np.ndarray(
        shape=(5, ), dtype='int32', buffer=weights_file.read(20))
print('Weights Header: ', weights_header)

Weights Header:  [       0        2        0 32013312        0]


## Build Model


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, Input, BatchNormalization, LeakyReLU, MaxPooling2D,
    UpSampling2D, Reshape, concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# ---------------------------
# Helper function to load weights (dummy for now)
# ---------------------------
def setWeights(filters, size, count, input_tensor, weights_file, doBatch):
    """
    Generate dummy weights for convolution and batch normalization layers.
    In practice, replace with reading from Darknet weights file.
    """
    prev_layer_shape = input_tensor.shape
    in_channels = prev_layer_shape[-1]

    # Convolution weights
    weights_shape = (size, size, in_channels, filters)
    conv_weights = [np.random.randn(*weights_shape).astype(np.float32)]

    # BatchNormalization weights
    if doBatch:
        bn_weight_list = [
            np.ones(filters, dtype=np.float32),   # gamma
            np.zeros(filters, dtype=np.float32),  # beta
            np.zeros(filters, dtype=np.float32),  # moving_mean
            np.ones(filters, dtype=np.float32)    # moving_variance
        ]
    else:
        bn_weight_list = []

    count += 1
    return conv_weights, bn_weight_list, count

# ---------------------------
# Model parameters
# ---------------------------
IMAGE_H, IMAGE_W = 416, 416
TRUE_BOX_BUFFER = 50
weights_file = None  # replace with actual weights file if available
weight_decay = 0.0005

# ---------------------------
# Model Inputs
# ---------------------------
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3), name='input_image')
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER, 4))

count = 0

# ---------------------------
# Layer 1
# ---------------------------
conv_weights, bn_weight_list, count = setWeights(16, 3, count, input_image, weights_file, True)
conv1 = Conv2D(16, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay),
               padding='same', use_bias=False, name='conv_1')
x = conv1(input_image)
if conv_weights:
    conv1.set_weights(conv_weights)

bn1 = BatchNormalization(name='norm_1')
x = bn1(x)
if bn_weight_list:
    bn1.set_weights(bn_weight_list)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ---------------------------
# Layer 2
# ---------------------------
conv_weights, bn_weight_list, count = setWeights(32, 3, count, x, weights_file, True)
conv2 = Conv2D(32, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay),
               padding='same', use_bias=False, name='conv_2')
x = conv2(x)
if conv_weights:
    conv2.set_weights(conv_weights)

bn2 = BatchNormalization(name='norm_2')
x = bn2(x)
if bn_weight_list:
    bn2.set_weights(bn_weight_list)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ---------------------------
# Layer 3
# ---------------------------
conv_weights, bn_weight_list, count = setWeights(64, 3, count, x, weights_file, True)
conv3 = Conv2D(64, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay),
               padding='same', use_bias=False, name='conv_3')
x = conv3(x)
if conv_weights:
    conv3.set_weights(conv_weights)

bn3 = BatchNormalization(name='norm_3')
x = bn3(x)
if bn_weight_list:
    bn3.set_weights(bn_weight_list)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

convd3 = x  # skip connection layer

# ---------------------------
# YOLO detection head
# ---------------------------
n_classes = 80
n_anchors = 3
yolo_conv = Conv2D(n_anchors * (n_classes + 5), (1,1), strides=(1,1), padding='same', name='yolo_head')
x = yolo_conv(x)
yolo_output = Reshape((x.shape[1], x.shape[2], n_anchors, n_classes + 5))(x)

# ---------------------------
# Create and save model
# ---------------------------
model = Model(inputs=[input_image], outputs=[yolo_output])

model.save('yolotest.keras')


In [ ]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 416, 416, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1 (Conv2D)                 │ (None, 416, 416, 16)   │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ norm_1 (BatchNormalization)     │ (None, 416, 416, 16)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 416, 416, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 208, 208, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 208, 208, 32)   │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ norm_2 (BatchNormalization)     │ (None, 208, 208, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 208, 208, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 104, 104, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 104, 104, 64)   │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ norm_3 (BatchNormalization)     │ (None, 104, 104, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 104, 104, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 52, 52, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ yolo_head (Conv2D)              │ (None, 52, 52, 255)    │        16,575 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 52, 52, 3, 85)  │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,495 (158.18 KB)

 Trainable params: 40,271 (157.31 KB)

 Non-trainable params: 224 (896.00 B)

In [ ]:
# Path to Darknet weights
weights_path = 'yolov3-tiny.weights'

# Open the weights file in binary mode
with open(weights_path, 'rb') as weights_file:
    # Read the first 5 integers in Darknet header (skip for now)
    _ = np.frombuffer(weights_file.read(5 * 4), dtype='int32')

    # ... your code to read layer weights using setWeights ...
    # For example:
    # conv_weights, bn_weight_list, count = setWeights(..., weights_file, ...)

    # After reading all layers, check remaining weights
    remaining_bytes = weights_file.read()  # read any leftover bytes
    remaining_weights = len(remaining_bytes) / 4  # each float32 = 4 bytes
    print('Read {} of {} from Darknet weights.'.format(count, count + remaining_weights))


Read 3 of 8858737.0 from Darknet weights.


###Load yolo model from file

In [ ]:
yolo = load_model('yolotest.keras')
# yolo = model

In [ ]:
# obj_threshold
# t1 = 0.6
t1 = 0.1
# nms_threshold
t2 = 0.5

def sigmoid(x):
        """sigmoid.

        # Arguments
            x: Tensor.

        # Returns
            numpy ndarray.
        """
        return 1 / (1 + np.exp(-x))

def process_feats(out, anchors, mask):
    """process output features.

    # Arguments
        out: Tensor (N, N, 3, 4 + 1 +80), output feature map of yolo.
        anchors: List, anchors for box.
        mask: List, mask for anchors.

    # Returns
        boxes: ndarray (N, N, 3, 4), x,y,w,h for per box.
        box_confidence: ndarray (N, N, 3, 1), confidence for per box.
        box_class_probs: ndarray (N, N, 3, 80), class probs for per box.
    """
    grid_h, grid_w, num_boxes = map(int, out.shape[1: 4])

    anchors = [anchors[i] for i in mask]
    anchors_tensor = np.array(anchors).reshape(1, 1, len(anchors), 2)

    # Reshape to batch, height, width, num_anchors, box_params.
    out = out[0]
    box_xy = sigmoid(out[..., :2])
    box_wh = np.exp(out[..., 2:4])
    box_wh = box_wh * anchors_tensor

    box_confidence = sigmoid(out[..., 4])
    box_confidence = np.expand_dims(box_confidence, axis=-1)
    box_class_probs = sigmoid(out[..., 5:])

    col = np.tile(np.arange(0, grid_w), grid_w).reshape(-1, grid_w)
    row = np.tile(np.arange(0, grid_h).reshape(-1, 1), grid_h)

    col = col.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    row = row.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    grid = np.concatenate((col, row), axis=-1)

    box_xy += grid
    box_xy /= (grid_w, grid_h)
    box_wh /= (416, 416)
    box_xy -= (box_wh / 2.)
    boxes = np.concatenate((box_xy, box_wh), axis=-1)

    return boxes, box_confidence, box_class_probs

def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x = boxes[:, 0]
    y = boxes[:, 1]
    w = boxes[:, 2]
    h = boxes[:, 3]

    areas = w * h
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x[i], x[order[1:]])
        yy1 = np.maximum(y[i], y[order[1:]])
        xx2 = np.minimum(x[i] + w[i], x[order[1:]] + w[order[1:]])
        yy2 = np.minimum(y[i] + h[i], y[order[1:]] + h[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 1)
        h1 = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= t2)[0]
        order = order[inds + 1]

    keep = np.array(keep)

    return keep

def filter_boxes(boxes, box_confidences, box_class_probs):
    """Filter boxes with object threshold.

    # Arguments
        boxes: ndarray, boxes of objects.
        box_confidences: ndarray, confidences of objects.
        box_class_probs: ndarray, class_probs of objects.

    # Returns
        boxes: ndarray, filtered boxes.
        classes: ndarray, classes for boxes.
        scores: ndarray, scores for boxes.
    """
    box_scores = box_confidences * box_class_probs
    box_classes = np.argmax(box_scores, axis=-1)
    box_class_scores = np.max(box_scores, axis=-1)
    pos = np.where(box_class_scores >= t1)

    boxes = boxes[pos]
    classes = box_classes[pos]
    scores = box_class_scores[pos]

    return boxes, classes, scores

def yolo_out(outs, shape):
    """Process output of yolo base net.

    # Argument:
        outs: output of yolo base net.
        shape: shape of original image.

    # Returns:
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
    """
    masks = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    anchors = [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45],
               [59, 119], [116, 90], [156, 198], [373, 326]]

    boxes, classes, scores = [], [], []

    for out, mask in zip(outs, masks):
        b, c, s = process_feats(out, anchors, mask)
        b, c, s = filter_boxes(b, c, s)
        boxes.append(b)
        classes.append(c)
        scores.append(s)

    boxes = np.concatenate(boxes)
    classes = np.concatenate(classes)
    scores = np.concatenate(scores)

    # Scale boxes back to original image shape.
    width, height = shape[0], shape[1]
    image_dims = [width, height, width, height]
    boxes = boxes * image_dims

    nboxes, nclasses, nscores = [], [], []
    for c in set(classes):
        inds = np.where(classes == c)
        b = boxes[inds]
        c = classes[inds]
        s = scores[inds]

        keep = nms_boxes(b, s)

        nboxes.append(b[keep])
        nclasses.append(c[keep])
        nscores.append(s[keep])

    if not nclasses and not nscores:
        return None, None, None

    boxes = np.concatenate(nboxes)
    classes = np.concatenate(nclasses)
    scores = np.concatenate(nscores)

    return boxes, classes, scores

def draw(image, boxes, scores, classes, all_classes):

  """Draw the boxes on the image.
    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
  """
#uncomment to use a custom font uploaded to this notebook directory
  fnt =ImageFont.truetype('./fonts/Ubuntu-Bold.ttf', 18)

  for box, score, cl in zip(boxes, scores, classes):

      x, y, w, h = box

      top = max(0, np.floor(x + 0.5).astype(int))
      left = max(0, np.floor(y + 0.5).astype(int))
      right = min(image.size[0], np.floor(x + w + 0.5).astype(int))
      bottom = min(image.size[1], np.floor(y + h + 0.5).astype(int))

      draw = ImageDraw.Draw(image)
      draw.rectangle([(top, left), (right, bottom)], outline=(0,255,0))#blue rectangle
      draw.text((top, left - 6), '{0} {1:.2f}'.format(all_classes[cl], score), font=fnt, fill=(0, 0, 255))
#      draw.text((top, left - 6), '{0} {1:.2f}'.format(all_classes[cl], score), fill=(0, 0, 255))
#         draw.rectangle([(left, top), (right, bottom)], outline=(0,0,255))#blue rectangle
      print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
      print('box coordinate x,y,w,h: {0}'.format(box))

  print()



###Load Image
Resize, reduce and expand image.

In [ ]:
# Create the directory if it doesn't exist
import os

if not os.path.exists('yolo3_tiny'):
    os.makedirs('yolo3_tiny')

# Change into the directory
%cd yolo3_tiny

# Verify the current path
!pwd


/content/yolo3_tiny/yolo3_tiny/yolo3_tiny
/content/yolo3_tiny/yolo3_tiny/yolo3_tiny


In [ ]:
from google.colab import files
uploaded = files.upload()  # Choose a file from your local PC

# Move it to images folder
for fname in uploaded.keys():
    os.rename(fname, os.path.join("images", fname))

input_image_name = list(uploaded.keys())[0]
print("Uploaded:", input_image_name)

# Process the image (same as above)
size = (416, 416)
image_src = Image.open("images/" + input_image_name)
orig_size = image_src.size
image_thumb = image_src.resize(size, Image.BICUBIC)
image_thumb.save("out/thumb_" + input_image_name, "JPEG")
image = np.array(image_thumb, dtype='float32') / 255.
image = np.expand_dims(image, axis=0)
print("Image shape:", image.shape)


Saving richard-brutyo-Sg3XwuEpybU-unsplash.jpg to richard-brutyo-Sg3XwuEpybU-unsplash.jpg
Uploaded: richard-brutyo-Sg3XwuEpybU-unsplash.jpg
Image shape: (1, 416, 416, 3)


###Get prediction output

In [ ]:
# Raw Prediction Output
output = yolo.predict(image)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step


In [ ]:
import os
import requests

# Make sure the file exists
if not os.path.exists("coco_classes.txt"):
    url = "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
    r = requests.get(url)
    with open("coco_classes.txt", "w") as f:
        f.write(r.text)
    print("Downloaded coco_classes.txt")

# Load class names
with open('coco_classes.txt') as f:
    class_names = f.readlines()
all_classes = [c.strip() for c in class_names]

print("Classes loaded:", all_classes[:10])  # print first 10 classes to check
/

Downloaded coco_classes.txt
Classes loaded: ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light']


()

In [ ]:
%cd ..
!pwd

/content/yolo3_tiny/yolo3_tiny
/content/yolo3_tiny/yolo3_tiny


In [ ]:
!pip install pycocotools


In [ ]:
!mkdir -p coco/images
%cd coco/images

# Download a small subset (val2017)
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
%cd ../../


Streaming output truncated to the last 5000 lines.
 extracting: val2017/000000231527.jpg  
 extracting: val2017/000000578922.jpg  
 extracting: val2017/000000062808.jpg  
 extracting: val2017/000000119038.jpg  
 extracting: val2017/000000114871.jpg  
 extracting: val2017/000000463918.jpg  
 extracting: val2017/000000365745.jpg  
 extracting: val2017/000000320425.jpg  
 extracting: val2017/000000481404.jpg  
 extracting: val2017/000000314294.jpg  
 extracting: val2017/000000335328.jpg  
 extracting: val2017/000000513688.jpg  
 extracting: val2017/000000158548.jpg  
 extracting: val2017/000000132116.jpg  
 extracting: val2017/000000415238.jpg  
 extracting: val2017/000000321333.jpg  
 extracting: val2017/000000081738.jpg  
 extracting: val2017/000000577584.jpg  
 extracting: val2017/000000346905.jpg  
 extracting: val2017/000000433980.jpg  
 extracting: val2017/000000228144.jpg  
 extracting: val2017/000000041872.jpg  
 extracting: val2017/000000117492.jpg  
 extracting: val2017/00000036

In [ ]:
!mkdir -p coco/annotations
%cd coco/annotations

# COCO 2017 validation annotations
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
%cd ../../


/content/yolo3_tiny/yolo3_tiny/coco/annotations
--2025-12-10 15:46:55--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.218.206, 3.5.29.23, 52.217.140.129, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.218.206|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  55.9MB/s    in 4.8s    

2025-12-10 15:47:00 (50.0 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val20

In [ ]:
# Create directories
!mkdir -p coco/images/val2017 coco/annotations

# Download a small sample of COCO 2017 validation images
!wget -O coco/images/val2017/sample1.jpg https://images.cocodataset.org/val2017/000000000139.jpg
!wget -O coco/images/val2017/sample2.jpg https://images.cocodataset.org/val2017/000000000285.jpg

# Download the small COCO annotations JSON for validation
!wget -O coco/annotations/instances_val2017.json \
https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco/instances_val2017.json


--2025-12-10 15:48:45--  https://images.cocodataset.org/val2017/000000000139.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.196.144, 52.216.34.1, 16.15.184.171, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.196.144|:443... connected.
ERROR: no certificate subject alternative name matches
	requested host name ‘images.cocodataset.org’.
To connect to images.cocodataset.org insecurely, use `--no-check-certificate'.
--2025-12-10 15:48:45--  https://images.cocodataset.org/val2017/000000000285.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.196.144, 52.216.34.1, 16.15.184.171, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.196.144|:443... connected.
ERROR: no certificate subject alternative name matches
	requested host name ‘images.cocodataset.org’.
To connect to images.cocodataset.org insecurely, use `--no-check-certificate'.
--2025-12-10 15:48:45--  https://raw.githubusercontent.com/pjreddie/da

In [ ]:
!ls -l coco/annotations/
!head -n 10 coco/annotations/instances_val2017.json


total 246996
drwxr-xr-x 2 root root      4096 Dec 10 15:47 annotations
-rw-r--r-- 1 root root 252907541 Jul 10  2018 annotations_trainval2017.zip
-rw-r--r-- 1 root root         0 Dec 10 15:48 instances_val2017.json
drwxr-xr-x 2 root root      4096 Dec 10 15:47 yolo_txt


In [ ]:
!mkdir -p coco/images/val2017
!mkdir -p coco/annotations/yolo_txt

# Download a few sample images
!wget -O coco/images/val2017/000000000139.jpg https://images.cocodataset.org/val2017/000000000139.jpg
!wget -O coco/images/val2017/000000000285.jpg https://images.cocodataset.org/val2017/000000000285.jpg

# Create a minimal COCO-style JSON with just these images
import json

small_coco = {
    "images": [
        {"id": 139, "width": 640, "height": 480, "file_name": "000000000139.jpg"},
        {"id": 285, "width": 640, "height": 480, "file_name": "000000000285.jpg"}
    ],
    "annotations": [
        {"id": 1, "image_id": 139, "category_id": 1, "bbox": [50, 50, 100, 200]},
        {"id": 2, "image_id": 285, "category_id": 18, "bbox": [120, 80, 200, 150]}
    ],
    "categories": [
        {"id": 1, "name": "person"},
        {"id": 18, "name": "dog"}
    ]
}

with open('coco/annotations/instances_val2017.json', 'w') as f:
    json.dump(small_coco, f)


--2025-12-10 15:50:14--  https://images.cocodataset.org/val2017/000000000139.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.31.10, 3.5.6.216, 16.15.216.165, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.31.10|:443... connected.
ERROR: no certificate subject alternative name matches
	requested host name ‘images.cocodataset.org’.
To connect to images.cocodataset.org insecurely, use `--no-check-certificate'.
--2025-12-10 15:50:15--  https://images.cocodataset.org/val2017/000000000285.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.31.10, 3.5.6.216, 16.15.216.165, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.31.10|:443... connected.
ERROR: no certificate subject alternative name matches
	requested host name ‘images.cocodataset.org’.
To connect to images.cocodataset.org insecurely, use `--no-check-certificate'.


In [ ]:
import json, os
from collections import defaultdict

image_dir = 'coco/images/val2017'
ann_file = 'coco/annotations/instances_val2017.json'
output_dir = 'coco/annotations/yolo_txt'
os.makedirs(output_dir, exist_ok=True)

with open(ann_file) as f:
    data = json.load(f)  # should work now

# rest of your conversion code...


In [ ]:
import os
import json
import numpy as np

# Directories
image_dir = 'coco/images/val2017'
ann_dir   = 'coco/annotations/yolo_txt'

# Make sure output folder exists
os.makedirs(ann_dir, exist_ok=True)


In [ ]:
from tensorflow.keras.models import load_model

yolo = load_model('yolotest.keras')  # your YOLOv3 tiny model


In [ ]:
def load_ground_truth(txt_file, img_size):
    boxes = []
    classes = []
    with open(txt_file, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            cx, cy, w, h = map(float, parts[1:])
            # Convert normalized [cx,cy,w,h] -> [x_min, y_min, x_max, y_max]
            x_min = (cx - w/2) * img_size[0]
            y_min = (cy - h/2) * img_size[1]
            x_max = (cx + w/2) * img_size[0]
            y_max = (cy + h/2) * img_size[1]
            boxes.append([x_min, y_min, x_max, y_max])
            classes.append(cls)
    return np.array(boxes), np.array(classes)


In [ ]:
def iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2]-box1[0])*(box1[3]-box1[1])
    box2Area = (box2[2]-box2[0])*(box2[3]-box2[1])
    return interArea / (box1Area + box2Area - interArea + 1e-6)


In [ ]:
def compute_metrics(pred_boxes, pred_classes, pred_scores, gt_boxes, gt_classes, iou_thresh=0.5):
    TP = 0
    FP = 0
    matched_gt = set()

    for pb, pc, ps in zip(pred_boxes, pred_classes, pred_scores):
        match = False
        for i, (gb, gc) in enumerate(zip(gt_boxes, gt_classes)):
            if i in matched_gt:
                continue
            if pc == gc and iou(pb, gb) >= iou_thresh:
                TP += 1
                matched_gt.add(i)
                match = True
                break
        if not match:
            FP += 1
    FN = len(gt_boxes) - len(matched_gt)
    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    return precision, recall


In [ ]:
from PIL import Image

iou_thresh = 0.5
all_precisions = []
all_recalls = []

for img_name in os.listdir(image_dir):
    if not img_name.endswith('.jpg'):
        continue
    img_path = os.path.join(image_dir, img_name)
    txt_path = os.path.join(ann_dir, os.path.splitext(img_name)[0]+'.txt')

    if not os.path.exists(txt_path):
        continue

    # Load image
    img = Image.open(img_path).resize((416, 416))
    img_array = np.array(img, dtype='float32')/255.
    img_array = np.expand_dims(img_array, axis=0)

    # YOLO predictions (dummy example, replace with your yolo_out)
    # Example format: boxes=[x_min,y_min,x_max,y_max], classes, scores
    # Here we assume your yolo_out function exists:
    pred_boxes, pred_classes, pred_scores = yolo_out(yolo(img_array), (416,416))

    # Load GT
    gt_boxes, gt_classes = load_ground_truth(txt_path, (416,416))

    # Compute metrics
    p, r = compute_metrics(pred_boxes, pred_classes, pred_scores, gt_boxes, gt_classes, iou_thresh)
    all_precisions.append(p)
    all_recalls.append(r)

print("Mean Precision:", np.mean(all_precisions))
print("Mean Recall:", np.mean(all_recalls))


Mean Precision: nan
Mean Recall: nan


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Prepare Model for Saving

In [ ]:
# Go up one directory level
%cd ..
# Save model for deployment
model.save("tinyYolo3")

In [ ]:
# Zip saved XOR model directory for download to local file system
!zip -r tinyYolo3.zip tinyYolo3

In [ ]:
# Get model input signature
model.input

In [ ]:
# Command to Authenticate with your Google account.
!gcloud auth login

In [ ]:
# Display the BEARER_TOKEN then copy and paste into the BEARER_TOKEN variable value in the next two cells
!gcloud auth print-access-token

In [ ]:
# Ref:
# https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models

import requests
import base64

PROJECT_ID='<INSERT_PROJECT_ID_HERE>'
ENDPOINT_ID='<INSERT_ENDPOINT_ID_HERE>'
BEARER_TOKEN = '<INSERT_BEARER_TOKEN_HERE>'
url = 'https://us-central1-aiplatform.googleapis.com/v1/projects/{}/locations/us-central1/endpoints/{}:predict'.format(PROJECT_ID, ENDPOINT_ID)
bearer = 'Bearer {}'.format(BEARER_TOKEN)

response = requests.post(url,
                  headers={'Content-Type': 'application/json',
                           'Authorization': bearer},
                  json={"instances": [{"b64": base64.b64encode(image).decode()}]}
                  )
print(response.json())
# print(response.json()['predictions'][0])